In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# setting parameters for papermill

consultant_name = 'sangeetha'
entry_num = '000'

In [3]:
from fsspec.implementations.local import LocalFileSystem
import re
import os
import sys
import dateutil.parser as parser
from ibots_db.schema import ConsultingReport
from ibots_db import update_all, load
from pathlib import Path

## Getting file paths to entries

In [18]:
fs = LocalFileSystem()
entry_file = f'data/entries/{consultant_name}/{entry_num}.txt'
assert Path(entry_file).exists(), f"Please make sure {entry_file} is available"
print(entry_file)

data/entries/sangeetha/000.txt
data/entries/sangeetha/000.txt


## Transform to dictionary

In [16]:
fun_split = lambda s: re.split(r'(Type:|Scholar:|Topic:|Date:|Content:)', s)
fun_key_add = lambda d: {key: d.get(key, '') for key in ['topic', 'scholar', 'date', 'content', 'type']}

entry = {'type':'', 'date':'', 'topic':'', 'content':'', 'scholar':''}
print(f"Getting dictionary from {entry_file}")
entry_text = open(entry_file, 'r', encoding='utf-8').read()
entry = {entry_splits[i].lower().strip(':'): entry_splits[i + 1].strip() for entry_splits in [fun_split(entry_text)] for i in range(1, len(entry_splits), 2)}
entry['date'] = parser.parse(entry['date']).strftime('%Y-%m-%d')
entry = fun_key_add(entry)
entry['consultant'] = consultant_name

Getting dictionary from data/entries/sangeetha/000.txt


## Validating entries

In [17]:
assert ConsultingReport(**entry), f"Could not validate entry from {entry_file}"